In [1]:
import os
import cv2
import numpy as np
import utils.ocrUtils as myutils

from imutils import contours

In [2]:
def cv_show(img):
    cv2.imshow("img", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

## 设置参数

In [3]:
image = "data/ocr/credit_card_01.png"
template = "data/ocr/ocr_a_reference.png"

FIRST_NUMBER = {"3": "American Express", 
                "4": "Visa", 
                "5": "MasterCard", 
                "6": "Discover Card"}

## 模板图像预处理

In [4]:
# 1. 读取图片，转化为灰度图
# 2. 对图像阈值进行处理

img = cv2.imread(template)
ref = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_, ref = cv2.threshold(ref, 10, 255, cv2.THRESH_BINARY_INV)

cv_show(ref)

## 计算模板图像轮廓

In [5]:
# cv2.findContours()函数接受的参数为二值图，即黑白的（不是灰度图）
# cv2.RETR_EXTERNAL只检测外轮廓
# cv2.CHAIN_APPROX_SIMPLE只保留终点坐标
# 返回的list中每个元素都是图像中的一个轮廓
# 对返回的轮廓从左到右，从上到下排序

_, refCnts, hierarchy = cv2.findContours(ref.copy(),
                                         cv2.RETR_EXTERNAL,
                                         cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(img, refCnts, -1, (0, 0, 255), 3)
# print(np.array(refCnts).shape)
refCnts = myutils.sort_contours(refCnts, method="left-to-right")[0]

In [6]:
# 遍历每一个轮廓
#   > 计算外接矩形并resize成合适大小
#   > 每一个数字对应一个模板

digits = {}

for (i, c) in enumerate(refCnts):
    (x, y, w, h) = cv2.boundingRect(c)
    roi = ref[y: y+h, x: x+w]
    roi = cv2.resize(roi, (57, 88))
    
    digits[i] = roi

## 信用卡图像预处理

In [7]:
# 初始化卷积核
rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 3))

image = cv2.imread(image)
image = myutils.resize(image, width=300)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

cv_show(gray)

In [8]:
# 礼帽操作，突出更明亮的区域
tophat = cv2.morphologyEx(gray, cv2.MORPH_TOPHAT, rectKernel)

cv_show(tophat)

In [9]:
gradX = cv2.Sobel(tophat, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=-1)
gradX = np.absolute(gradX)

(minVal, maxVal) = (np.min(gradX), np.max(gradX))
gradX = (255 * ((gradX - minVal) / (maxVal - minVal)))
gradX = gradX.astype("uint8")

cv_show(gradX)

In [10]:
# 通过闭操作（先膨胀，再腐蚀），将数字连在一起
gradX = cv2.morphologyEx(gradX, cv2.MORPH_CLOSE, rectKernel)

cv_show(gradX)

In [11]:
#THRESH_OTSU会自动寻找合适的阈值，适合双峰，需把阈值参数设置为0

_, thresh = cv2.threshold(gradX, 0, 255, 
                       cv2.THRESH_BINARY | cv2.THRESH_OTSU)

cv_show(thresh)

In [12]:
# 再来一个闭操作
sqKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))

thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, sqKernel)
cv_show(thresh)

## 信用卡轮廓计算

In [13]:
_, threshCnts, hierarchy = cv2.findContours(thresh.copy(), 
                                            cv2.RETR_EXTERNAL,
                                            cv2.CHAIN_APPROX_SIMPLE)

cnts = threshCnts
cur_img = image.copy()

cv2.drawContours(cur_img, cnts, -1, (0, 0, 255), 3)
cv_show(cur_img)

In [14]:
# 遍历轮廓
#    > 计算矩形
#    > 选择合适的区域，根据实际任务来，这里基本都是四个数字一组

locs = []

for (i, c) in enumerate(cnts):
    (x, y, w, h) = cv2.boundingRect(c)
    ar = w / float(h)
    
    if ar > 2.5 and ar < 4.0:
        if(w > 40 and w < 55) and (h > 10 and h < 20):
            locs.append((x, y, w, h))

# 将符合得轮廓从左到右排序
locs = sorted(locs, key=lambda x:x[0])

## 通过模板匹配找到信用卡上得数字

In [15]:
outputs = []

for (i, (gX, gY, gW, gH)) in enumerate(locs):
    groupOutput = []
    
    # 根据坐标提取每一个组
    group = gray[gY - 5: gY + gH + 5, gX - 5: gX + gW + 5]
    
    # 图像预处理
    _, group = cv2.threshold(group, 0, 255,
                          cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    
    # 计算每一组得轮廓
    _, digitCnts, hierarchy = cv2.findContours(group.copy(),
                                               cv2.RETR_EXTERNAL,
                                               cv2.CHAIN_APPROX_SIMPLE)
    digitCnts = contours.sort_contours(digitCnts,
                                       method="left-to-right")[0]
    
    # 计算每一组中的每一个数值
    for c in digitCnts:
        # 找到当前数值的轮廓，resize成合适的大小
        (x, y, w, h) = cv2.boundingRect(c)
        roi = group[y: y+h, x: x+w]
        roi = cv2.resize(roi, (57, 88))
        
        # 计算匹配得分
        scores = []
        
        # 在模板中计算每一个得分
        for (digit, digitROI) in digits.items():
            # 模板匹配
            result = cv2.matchTemplate(roi, digitROI, cv2.TM_CCOEFF)
            (_, score, _, _) = cv2.minMaxLoc(result)
        
            scores.append(score)
        
        # 得到最合适的数字
        groupOutput.append(str(np.argmax(scores)))
    
    # 画出来
    cv2.rectangle(image, (gX - 5, gY - 5),
                  (gX + gW + 5, gY + gH + 5),
                  (0, 0, 255), 1)
    cv2.putText(image, "".join(groupOutput), (gX, gY - 15),
                cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 0, 255), 2)
    
    # 得到结果
    outputs.extend(groupOutput)

In [16]:
print("Credit Card Type: {}".format(FIRST_NUMBER[outputs[0]]))
print("Credit Card #: {}".format("".join(outputs)))

cv_show(image)

Credit Card Type: Visa
Credit Card #: 4000123456789010
